In [ ]:
# نصب پکیج‌های لازم
!pip install --upgrade rouge_score datasets transformers torch

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━

In [ ]:
!pip install --upgrade transformers==4.44.2 datasets==2.21.0 rouge_score torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 81.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.1
    Uninstalling transformers-4.53.1:
      Successfully uninstalled transformers-4.53.1
  Attempting uninstall: datasets
    Found existing installation: datasets 3.6.0
    Uninstal

In [ ]:

import torch
import numpy as np
import os
from rouge_score import rouge_scorer
from transformers import BartForConditionalGeneration, BartTokenizer
from datasets import load_dataset


In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = BartForConditionalGeneration.from_pretrained("facebook/bart-base").to(device)
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

dataset = load_dataset("abisee/cnn_dailymail", "3.0.0")
val_dataset = dataset['validation'].select(range(700))
train_dataset = dataset['train'].select(range(3000))
test_dataset = dataset['test'].select(range(200))

def preprocess_data(example):
    article = example['article']
    summary = example['highlights']
    inputs = tokenizer(article, max_length=512, truncation=True, padding="max_length", return_tensors="pt")
    outputs = tokenizer(summary, max_length=150, truncation=True, padding="max_length", return_tensors="pt")
    return {
        "input_ids": inputs["input_ids"].squeeze(),
        "attention_mask": inputs["attention_mask"].squeeze(),
        "labels": outputs["input_ids"].squeeze(),
        "article": article,
        "highlights": summary
    }

val_dataset = val_dataset.map(preprocess_data)
train_dataset = train_dataset.map(preprocess_data)
test_dataset = test_dataset.map(preprocess_data)

val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"], output_all_columns=True)
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"], output_all_columns=True)
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"], output_all_columns=True)

print(val_dataset[0]["input_ids"].shape)
print(type(val_dataset[0]["input_ids"]))
print("Article exists:", "article" in val_dataset[0])
print("Highlights exists:", "highlights" in val_dataset[0])

Using device: cuda


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/700 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

torch.Size([512])
<class 'torch.Tensor'>
Article exists: True
Highlights exists: True


In [ ]:
import os
os.environ["WANDB_MODE"] = "disabled"
os.environ["WANDB_DISABLED"] = "true"

import wandb
print(wandb.__version__)
print(wandb.run)

0.20.1
None


In [ ]:
import os
os.environ["WANDB_MODE"] = "disabled"

from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir="./bart-summarizer-final",
    num_train_epochs=7,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    evaluation_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=True,
    save_total_limit=2,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# آموزش مدل با Early Stopping
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss,Validation Loss
500,1.009600,0.703391
1000,0.806900,0.703704
1500,0.783200,0.698573
2000,0.618400,0.725979
2500,0.514000,0.760027
3000,0.524600,0.749106
3500,0.431000,0.774560
4000,0.356300,0.805544
4500,0.373500,0.800901
5000,0.322300,0.815752


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file

TrainOutput(global_step=5250, training_loss=0.7819913431803386, metrics={'train_runtime': 2333.4668, 'train_samples_per_second': 8.999, 'train_steps_per_second': 2.25, 'total_flos': 6402232811520000.0, 'train_loss': 0.7819913431803386, 'epoch': 7.0})

In [ ]:

def compute_metrics(eval_pred):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    predictions, labels = eval_pred


    predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True)


    rouge1_scores = []
    rouge2_scores = []
    rougeL_scores = []

    for pred, ref in zip(predictions, labels):
        scores = scorer.score(ref, pred)
        rouge1_scores.append(scores['rouge1'].fmeasure)
        rouge2_scores.append(scores['rouge2'].fmeasure)
        rougeL_scores.append(scores['rougeL'].fmeasure)

    return {
        "rouge1": np.mean(rouge1_scores),
        "rouge2": np.mean(rouge2_scores),
        "rougeL": np.mean(rougeL_scores)
    }

In [ ]:
# تابع خلاصه‌سازی
def summarize_text(text, model, tokenizer, max_length=150):
    inputs = tokenizer(text, max_length=512, truncation=True, padding="max_length", return_tensors="pt")
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    summary_ids = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=max_length,
        min_length=40,
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True
    )
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

def evaluate_model(model, tokenizer, dataset):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    predictions = []
    references = []

    model = model.to(device)

    for example in dataset:
        pred = summarize_text(example["article"], model, tokenizer)
        ref = example["highlights"]
        predictions.append(pred)
        references.append(ref)


    rouge_scores = {
        'rouge1': {'precision': [], 'recall': [], 'fmeasure': []},
        'rouge2': {'precision': [], 'recall': [], 'fmeasure': []},
        'rougeL': {'precision': [], 'recall': [], 'fmeasure': []}
    }

    for pred, ref in zip(predictions, references):
        scores = scorer.score(ref, pred)
        for metric in ['rouge1', 'rouge2', 'rougeL']:
            rouge_scores[metric]['precision'].append(scores[metric].precision)
            rouge_scores[metric]['recall'].append(scores[metric].recall)
            rouge_scores[metric]['fmeasure'].append(scores[metric].fmeasure)


    for metric in rouge_scores:
        rouge_scores[metric]['precision'] = np.mean(rouge_scores[metric]['precision'])
        rouge_scores[metric]['recall'] = np.mean(rouge_scores[metric]['recall'])
        rouge_scores[metric]['fmeasure'] = np.mean(rouge_scores[metric]['fmeasure'])

    return rouge_scores


In [ ]:
# ارزیابی روی دیتاست
rouge_scores = evaluate_model(model, tokenizer, test_dataset)
print("ROUGE Scores:", rouge_scores)

ROUGE Scores: {'rouge1': {'precision': np.float64(0.29272023430880034), 'recall': np.float64(0.38529435439062854), 'fmeasure': np.float64(0.325581251464433)}, 'rouge2': {'precision': np.float64(0.10200851876324048), 'recall': np.float64(0.13802444217357093), 'fmeasure': np.float64(0.11470078304490441)}, 'rougeL': {'precision': np.float64(0.20590357520341362), 'recall': np.float64(0.2732477110973972), 'fmeasure': np.float64(0.22976141479789752)}}


In [ ]:

for metric, scores in rouge_scores.items():
    print(f"{metric}:")
    print(f"Precision: {scores['precision']}")
    print(f"Recall: {scores['recall']}")
    print(f"F1 Score: {scores['fmeasure']}")
    print()

rouge1:
Precision: 0.29272023430880034
Recall: 0.38529435439062854
F1 Score: 0.325581251464433

rouge2:
Precision: 0.10200851876324048
Recall: 0.13802444217357093
F1 Score: 0.11470078304490441

rougeL:
Precision: 0.20590357520341362
Recall: 0.2732477110973972
F1 Score: 0.22976141479789752



In [ ]:
# ذخیره مدل و توکنایزر
save_dir = "./bart-summarizer-final"
os.makedirs(save_dir, exist_ok=True)
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


('./bart-summarizer-final/tokenizer_config.json',
 './bart-summarizer-final/special_tokens_map.json',
 './bart-summarizer-final/vocab.json',
 './bart-summarizer-final/merges.txt',
 './bart-summarizer-final/added_tokens.json')

A dozen or more metal implements are arranged in neat rows on the table next to the bed. They look like something Frankenstein might have had applied to his monster's neck. I begin to feel nervous. What on earth have I got myself into? The pristine white room adds to the feeling that I'm about to undergo major surgery. But I'm not. I'm here to try the wackiest face treatment to hit the beauty scene: the tuning fork facial. Offered at the upmarket Hale Clinic in London and other clinics nationwide, the scary sounding tuning forks are pieces of metal that make different pitched sounds when struck and made to vibrate. Tuned in: Marianne is given the tuning fork treatment . When they are placed on specific parts of the face - the neck, the jowls, forehead and around the eyes - vibrations are said to tighten sagging skin, eradicate dark circles, lessen the appearance of wrinkles and stimulate collagen. The results mean that your face looks younger, brighter and more toned. Some say it's the non-jab equivalent of Botox. The treatment comes from the U.S. - where else? Advocates claim the 'sound waves of the forks vibrate and travel deeply into the body along energy pathways, affecting human physiology and reaching places not easily accessed'. So far, so New Age gibberish. There seems to be no scientific proof that tuning forks can make you look younger, prettier or more rested. It sounds nutty - the kind of thing practised by people who like panpipes and hemp seeds. The UK beauty industryÂ employs more aÂ million people andÂ is worth Â£17 bn . I prefer my facials to involve peels, not strange pieces of metal. But while I may be sceptical, I am also, at 37, willing to try anything. Over the past few years, frown lines have begun to appear on my forehead, and my face appears increasingly pale, puffy and bloated. I explain all this to therapist Samara Reid, who's sympathetic. 'What's wonderful about these facials is they improve the appearance of the skin, but also treat the cause of the problem,' she says. 'If you're pale, you probably have poor circulation. The forks can boost blood flow, which will give you a healthier colour.' Samara has been a facial acupuncturist for 20 years, but she has started using the forks on their own and in conjunction with needles. 'I can see my jawline for the first time in a while, and my skin looks plumper and pink, rather than white' The therapies work on the same principle of Chinese acupuncture, which follows the idea that our body's energy - or 'Qi' - flows through 12 main channels called meridians. If this flow is interrupted, it can cause ill health and affect our appearance, causing premature ageing, dry skin and eye bags. Inserting needles or vibrating tuning fork on these trigger points supposedly restores the flow of energy, improving fine lines and circulation, leading to a clearer, rejuvenated complexion. Samara uses them together for a more powerful effect, but for the needle-phobic the forks can be used on their own. Certain forks are designed for different parts of the face and the body - there are dozens of them, with names such as 'Sun', 'Earth' and 'New Moon'. The physical vibrations have an effect on the body, while the sound from the forks is simultaneously soothing. Samara pings one of the forks and puts it up to my ear. The sound is magical, like a xylophone from outer space. She asks me to get on the bed and remove my socks. She explains she will tap my feet with the forks and apply needles to 'bring the energy down for grounding' (whatever that means) before starting on the face and getting the energy flowing all around my body. Samara uses a 'Gold Ohm Unison' tuning fork. She strikes it against a hard disc strapped on her leg, which creates a vibration, before putting the base of the fork on various points on my feet. I can feel a gentle, deep vibration travel through my feet and halfway up my legs. It's a strange, tingly sensation, and builds as she places the fork on more points on my feet. Within two minutes I feel as if I'm floating. She then puts needles in between my toes to 'cleanse the system'. They don't hurt at all. Then she moves to my face. She feels around my jawline and says she can feel I have raised glands. She asks if I get a lot of sore throats, and I say yes. 'The vibration from the forks can help the lymph glands to drain, which will help bring down the swelling,' she says, adding that all she's doing is 'clearing blockages', which will help everything work better. Inserting needles or vibrating tuning fork on these trigger points supposedly restores the flow of energy, improving fine lines and circulation, leading to a clearer, rejuvenated complexion . Marinna finds the physical vibrations have an effect on the body, while the sound from the forks is simultaneously soothing . She also explains that she uses tuning forks to help tighten loose muscle around the jawline. As she puts the base of the 'Earth' fork around my jawline, the vibrations go deep into my head. It feels odd, but also rather lovely. She then puts the forks up to my ears and I hear a clear, deep, bell-like sound that feels as if it's cleaning my mind. All my normal chattering thoughts are gone for a moment. Samara, who is wafting around the room ethereally, as New Aged therapists are wont to do, explains in her soft, slow voice that different forks have different sounds, varying from deep and resonant to high-pitched. She says the 'New Moon' fork is good for wrinkles and crow's feet, because it relaxes the skin. But when she starts talking about how the sounds will 'bring me into alignment with the cycles and rhythms of the cosmos', it feels a step too far. She found the effect was less pronounced the following day - although she still looked better . I begin to feel it is all ridiculous. But then she moves up my face. Samara tells me I have deep lines between my nose and mouth and my cheeks are puffy. She asks if I get a lot of sinus problems. Again, I say yes. She's more thorough than some GPs I've seen. Samara presses the forks along the side of my nose, where the vibrations should help drain the sinuses, and then my eyes, where apparently they will help reduce dark circles. Finally, Samara comes to my forehead. She tells me I have a line forming between my eyebrows, known as the 'suspended sword', which can be caused by fear and concentration. 'Do you frown when you are concentrating?' she asks. I do. 'And do you find yourself feeling frightened quite often?' she asks. Yes! This woman is a mind-reader. And then we're done. It feels as if I've been lying there for a minute, but Samara tells me it's nearly an hour. I feel spaced out. I'm so relaxed it's hard to speak. Apparently, this is normal, the result of feel-good endorphins created when pressure points are stimulated. I'm pleasantly surprised - in the mirror, my face looks less like the Pilsbury Doughboy. I can see my jawline for the first time in a while, and my skin looks plumper and pink, rather than white. The next morning, I take a long, hard look at my face. The effect isn't as pronounced as the day before, but my skin looks smoother and I have more colour. Samara recommends 12 sessions, but most people see a difference after just three or four. At Â£90 for an hour, it's not cheap, but it's reminded me of a basic beauty truth: a glowing complexion isn't something to be obtained in an expensive cream - it comes from being healthy inside and out. Samara packs me off with some good advice: drink fresh ginger tea to boost my immune system and try to avoid dairy, which doesn't help if you're prone to congestion, coughs and colds. Maybe I am the hemp seed type after all. I have booked a second appointment. Samara REID, the Hale Clinic, London (020 7631 0156). Experts recommend 12 sessions. At Â£90 for an hour, it's not cheap .





Marianne Power triedÂ the tuning fork facial atÂ Hale Clinic in London .
Utilises tuning forks that make different pitched sounds when struck .
Says afterwards her skin looks plumper and pink, rather than white .

In [ ]:


# تست خلاصه‌سازی با متن نمونه
article = "A dozen or more metal implements are arranged in neat rows on the table next to the bed. They look like something Frankenstein might have had applied to his monster's neck. I begin to feel nervous. What on earth have I got myself into? The pristine white room adds to the feeling that I'm about to undergo major surgery. But I'm not. I'm here to try the wackiest face treatment to hit the beauty scene: the tuning fork facial. Offered at the upmarket Hale Clinic in London and other clinics nationwide, the scary sounding tuning forks are pieces of metal that make different pitched sounds when struck and made to vibrate. Tuned in: Marianne is given the tuning fork treatment . When they are placed on specific parts of the face - the neck, the jowls, forehead and around the eyes - vibrations are said to tighten sagging skin, eradicate dark circles, lessen the appearance of wrinkles and stimulate collagen. The results mean that your face looks younger, brighter and more toned. Some say it's the non-jab equivalent of Botox. The treatment comes from the U.S. - where else? Advocates claim the 'sound waves of the forks vibrate and travel deeply into the body along energy pathways, affecting human physiology and reaching places not easily accessed'. So far, so New Age gibberish. There seems to be no scientific proof that tuning forks can make you look younger, prettier or more rested. It sounds nutty - the kind of thing practised by people who like panpipes and hemp seeds. The UK beauty industryÂ employs more aÂ million people andÂ is worth Â£17 bn . I prefer my facials to involve peels, not strange pieces of metal. But while I may be sceptical, I am also, at 37, willing to try anything. Over the past few years, frown lines have begun to appear on my forehead, and my face appears increasingly pale, puffy and bloated. I explain all this to therapist Samara Reid, who's sympathetic. 'What's wonderful about these facials is they improve the appearance of the skin, but also treat the cause of the problem,' she says. 'If you're pale, you probably have poor circulation. The forks can boost blood flow, which will give you a healthier colour.' Samara has been a facial acupuncturist for 20 years, but she has started using the forks on their own and in conjunction with needles. 'I can see my jawline for the first time in a while, and my skin looks plumper and pink, rather than white' The therapies work on the same principle of Chinese acupuncture, which follows the idea that our body's energy - or 'Qi' - flows through 12 main channels called meridians. If this flow is interrupted, it can cause ill health and affect our appearance, causing premature ageing, dry skin and eye bags. Inserting needles or vibrating tuning fork on these trigger points supposedly restores the flow of energy, improving fine lines and circulation, leading to a clearer, rejuvenated complexion. Samara uses them together for a more powerful effect, but for the needle-phobic the forks can be used on their own. Certain forks are designed for different parts of the face and the body - there are dozens of them, with names such as 'Sun', 'Earth' and 'New Moon'. The physical vibrations have an effect on the body, while the sound from the forks is simultaneously soothing. Samara pings one of the forks and puts it up to my ear. The sound is magical, like a xylophone from outer space. She asks me to get on the bed and remove my socks. She explains she will tap my feet with the forks and apply needles to 'bring the energy down for grounding' (whatever that means) before starting on the face and getting the energy flowing all around my body. Samara uses a 'Gold Ohm Unison' tuning fork. She strikes it against a hard disc strapped on her leg, which creates a vibration, before putting the base of the fork on various points on my feet. I can feel a gentle, deep vibration travel through my feet and halfway up my legs. It's a strange, tingly sensation, and builds as she places the fork on more points on my feet. Within two minutes I feel as if I'm floating. She then puts needles in between my toes to 'cleanse the system'. They don't hurt at all. Then she moves to my face. She feels around my jawline and says she can feel I have raised glands. She asks if I get a lot of sore throats, and I say yes. 'The vibration from the forks can help the lymph glands to drain, which will help bring down the swelling,' she says, adding that all she's doing is 'clearing blockages', which will help everything work better. Inserting needles or vibrating tuning fork on these trigger points supposedly restores the flow of energy, improving fine lines and circulation, leading to a clearer, rejuvenated complexion . Marinna finds the physical vibrations have an effect on the body, while the sound from the forks is simultaneously soothing . She also explains that she uses tuning forks to help tighten loose muscle around the jawline. As she puts the base of the 'Earth' fork around my jawline, the vibrations go deep into my head. It feels odd, but also rather lovely. She then puts the forks up to my ears and I hear a clear, deep, bell-like sound that feels as if it's cleaning my mind. All my normal chattering thoughts are gone for a moment. Samara, who is wafting around the room ethereally, as New Aged therapists are wont to do, explains in her soft, slow voice that different forks have different sounds, varying from deep and resonant to high-pitched. She says the 'New Moon' fork is good for wrinkles and crow's feet, because it relaxes the skin. But when she starts talking about how the sounds will 'bring me into alignment with the cycles and rhythms of the cosmos', it feels a step too far. She found the effect was less pronounced the following day - although she still looked better . I begin to feel it is all ridiculous. But then she moves up my face. Samara tells me I have deep lines between my nose and mouth and my cheeks are puffy. She asks if I get a lot of sinus problems. Again, I say yes. She's more thorough than some GPs I've seen. Samara presses the forks along the side of my nose, where the vibrations should help drain the sinuses, and then my eyes, where apparently they will help reduce dark circles. Finally, Samara comes to my forehead. She tells me I have a line forming between my eyebrows, known as the 'suspended sword', which can be caused by fear and concentration. 'Do you frown when you are concentrating?' she asks. I do. 'And do you find yourself feeling frightened quite often?' she asks. Yes! This woman is a mind-reader. And then we're done. It feels as if I've been lying there for a minute, but Samara tells me it's nearly an hour. I feel spaced out. I'm so relaxed it's hard to speak. Apparently, this is normal, the result of feel-good endorphins created when pressure points are stimulated. I'm pleasantly surprised - in the mirror, my face looks less like the Pilsbury Doughboy. I can see my jawline for the first time in a while, and my skin looks plumper and pink, rather than white. The next morning, I take a long, hard look at my face. The effect isn't as pronounced as the day before, but my skin looks smoother and I have more colour. Samara recommends 12 sessions, but most people see a difference after just three or four. At Â£90 for an hour, it's not cheap, but it's reminded me of a basic beauty truth: a glowing complexion isn't something to be obtained in an expensive cream - it comes from being healthy inside and out. Samara packs me off with some good advice: drink fresh ginger tea to boost my immune system and try to avoid dairy, which doesn't help if you're prone to congestion, coughs and colds. Maybe I am the hemp seed type after all. I have booked a second appointment. Samara REID, the Hale Clinic, London (020 7631 0156). Experts recommend 12 sessions. At Â£90 for an hour, it's not cheap ."
summary = summarize_text(article, model, tokenizer)
print("Summary:", summary)


Summary: A dozen or more metal implements are arranged in neat rows on table next to bed.
Tuning forks are made to make different pitched sounds when struck.
Some say they are the non-jab equivalent of Botox.
